# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,-1.34,91,19,1.49,CL,2024-05-06
1,1,praia da vitoria,38.7333,-27.0667,19.14,94,75,12.65,PT,2024-05-06
2,2,mahajanga,-15.7167,46.3167,26.01,61,20,6.17,MG,2024-05-06
3,3,nelson,-41.2833,173.2833,10.72,70,8,0.70,NZ,2024-05-06
4,4,grytviken,-54.2811,-36.5092,0.60,83,69,4.34,GS,2024-05-06


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    color = 'City',
    alpha = 0.75,
    tiles = "EsriNatGeo",
    size ='Humidity',
    
    
    
)
# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_criteria = city_data_df[(city_data_df['Max Temp'] <= 18) &
                                   (city_data_df['Humidity'] > 40) &
                                   (city_data_df['Cloudiness'] <= 30) &
                                   (city_data_df['Wind Speed'] <= 15)]
# Drop any rows with null values
# YOUR CODE HERE
ideal_criteria_df = ideal_criteria.dropna()
# Display sample data
# YOUR CODE 
ideal_criteria_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,-1.34,91,19,1.49,CL,2024-05-06
3,3,nelson,-41.2833,173.2833,10.72,70,8,0.70,NZ,2024-05-06
14,14,iqaluit,63.7506,-68.5145,-1.15,64,20,4.63,CA,2024-05-06
15,15,margaret river,-33.9500,115.0667,12.39,87,8,3.32,AU,2024-05-06
20,20,bredasdorp,-34.5322,20.0403,14.78,66,0,1.41,ZA,2024-05-06
...,...,...,...,...,...,...,...,...,...,...
563,563,dubbo,-32.2500,148.6167,13.07,71,0,6.17,AU,2024-05-06
581,581,motueka,-41.1333,173.0167,11.72,57,22,0.89,NZ,2024-05-06
583,583,saint-denis,48.9167,2.3333,12.45,81,0,2.57,FR,2024-05-06
585,585,puerto deseado,-47.7503,-65.8938,4.06,59,26,4.73,AR,2024-05-06


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_criteria_df[['City', 'Country', 'Lat','Lng', "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ''
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto natales,CL,-51.7236,-72.4875,91,
3,nelson,NZ,-41.2833,173.2833,70,
14,iqaluit,CA,63.7506,-68.5145,64,
15,margaret river,AU,-33.9500,115.0667,87,
20,bredasdorp,ZA,-34.5322,20.0403,66,
...,...,...,...,...,...,...
563,dubbo,AU,-32.2500,148.6167,71,
581,motueka,NZ,-41.1333,173.0167,57,
583,saint-denis,FR,48.9167,2.3333,81,
585,puerto deseado,AR,-47.7503,-65.8938,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  #YOUR CODE HERE
categories = ('internet_access.free','accommodation.hotel')
params = {'radius': radius, 'categories': categories , 'apiKey': geoapify_key
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    Lat = row["Lat"]
    Lng = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"# YOUR CODE HERE
    params["bias"] = f"proximity:{Lng},{Lat}"# YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)# YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json()# YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto natales - nearest hotel: Dorotea Patagonia Hostel
nelson - nearest hotel: Skylight Villa
iqaluit - nearest hotel: Frobisher Inn
margaret river - nearest hotel: Margaret River Hotel
bredasdorp - nearest hotel: Victoria Hotel
qaqortoq - nearest hotel: Hotel Qaqortoq
plettenberg bay - nearest hotel: Milkwood Manor
mossel bay - nearest hotel: Protea Hotel Mossel Bay
kittila - nearest hotel: Hotelli Kittilä
fuli - nearest hotel: 恒大酒店
mzimba - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
yangquan - nearest hotel: 7天酒店
egilsstadir - nearest hotel: Sólbrekka 16 Guesthouse
zaragoza - nearest hotel: Hotel Pilar Plaza
punta umbria - nearest hotel: Pato Amarillo
anadyr - nearest hotel: Гостевой дом
aljezur - nearest hotel: Amazigh Design Hostel
nuuk - nearest hotel: Nordbo
lubango - nearest hotel: Novo Hotel
el calafate - nearest hotel: Hostería Las Avutardas
collie - nearest hotel: Federal Hotel
chiredzi - nearest hotel: Nesbitt Arms Hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto natales,CL,-51.7236,-72.4875,91,Dorotea Patagonia Hostel
3,nelson,NZ,-41.2833,173.2833,70,Skylight Villa
14,iqaluit,CA,63.7506,-68.5145,64,Frobisher Inn
15,margaret river,AU,-33.9500,115.0667,87,Margaret River Hotel
20,bredasdorp,ZA,-34.5322,20.0403,66,Victoria Hotel
...,...,...,...,...,...,...
563,dubbo,AU,-32.2500,148.6167,71,Blue Gum Motel
581,motueka,NZ,-41.1333,173.0167,57,Vineyard Tourist Units
583,saint-denis,FR,48.9167,2.3333,81,Tribe
585,puerto deseado,AR,-47.7503,-65.8938,59,Los Olmos


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",    
    size = "Humidity",
    color = "City",
    
    hover_cols=["Country", "Hotel Name"]
)

# Display the map
# YOUR CODE HERE
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)